# Proof-of-concept: Further Communications Protocols

This notebook experiments further communication algorithms other than MAPPO (our baseline).

In [1]:
import sys
import os
ROOT = os.path.join(os.path.dirname(os.curdir), '..')
SIMPLEGRID_PATH = os.path.join(ROOT, 'gym-simplegrid', 'gym_simplegrid', 'envs')
sys.path.append(ROOT)
sys.path.append(SIMPLEGRID_PATH)
from simple_grid import SimpleGridEnv
import tensorflow as tf
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt

## Recall Environment Settings

In [2]:
# Cell values
FREE: int = 0
OBSTACLE_SOFT: int = 1
OBSTACLE_HARD: int = 2
AGENT: int = 3
TARGET: int = 4

In [3]:
# Partial Observability Definition
observation_radius = 2

In [4]:
from constants import ACTION_SPACE, REWARDS, LEADER_MESSAGE_SIZE
print("ACTION_SPACE:", [(action.name, action.value )for action in ACTION_SPACE])
print("REWARDS:", [(reward.name, reward.value) for reward in REWARDS])
print("LEADER_MESSAGE_SIZE:", LEADER_MESSAGE_SIZE)

ACTION_SPACE: [('UP', (-1, 0)), ('DOWN', (1, 0)), ('LEFT', (0, -1)), ('RIGHT', (0, 1)), ('UP_LEFT', (-1, -1)), ('UP_RIGHT', (-1, 1)), ('DOWN_LEFT', (1, -1)), ('DOWN_RIGHT', (1, 1)), ('STAY', (0, 0))]
REWARDS: [('SOFT_OBSTACLE', -10), ('HARD_OBSTACLE', -50), ('TARGET', 50), ('STEP', -1), ('STAY', -3)]
LEADER_MESSAGE_SIZE: 8


## DIAL (Differentiable Inter-Agent Learning) Algorithm

### Design the Network's Structure

In [5]:
def build_leader_model(observation_radius: int, leader_message_size: int):
    '''
        LEADER MODEL: Observe the state and Make its own decision + Send a message to the FOLLOWER

        Args:
            Input shape: LEADER Observable State

        Returns:
            Output shape: LEADER Action Probabilities + Leader's Message
    '''
    # Shared layers
    inputs = tf.keras.layers.Input(shape=(1 + observation_radius * 2, 1 + observation_radius * 2))
    x = tf.keras.layers.Reshape((-1, 1))(inputs) # Flatten spatial grid to sequence: (batch_size, time_steps = flattened size, features = 1)
    x = tf.keras.layers.GRU(leader_message_size, activation='softmax', return_sequences=False)(x)
    x = tf.keras.layers.Dense(64, activation='softmax')(x)

    # Action head
    action_logits = tf.keras.layers.Dense(len(ACTION_SPACE), activation='linear', name="action_logits")(x)

    # Message head
    message = tf.keras.layers.Dense(leader_message_size, activation='sigmoid', name="message")(x)

    # Value Prediction
    value = tf.keras.layers.Dense(1)(x)  # Value prediction

    # Model
    leader_model = tf.keras.Model(inputs=inputs, outputs=[action_logits, message, value])
    leader_model.summary()
    return leader_model
leader_model = build_leader_model(observation_radius, LEADER_MESSAGE_SIZE)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 5, 5)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 25, 1)     │          0 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gru (GRU)           │ (None, 8)         │        264 │ reshape[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │        576 │ gru[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ action_logits       │ (None, 9)         │        585 │ dense[0][0]       │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ message (Dense)     │ (None, 8)         │        520 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 1)         │         65 │ dense[0][0]       │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,010 (7.85 KB)

 Trainable params: 2,010 (7.85 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
def build_follower_model(observation_radius: int, leader_message_size: int):
    '''
        FOLLOWER MODEL: Observe the state + Receive a message from the LEADER + Make its own decision

        Args:
            Input shape: FOLLOWER Observable State + Message from LEADER
        
        Returns:
            Output shape: FOLLOWER Action Probabilities
    '''
    # Shared Input layers
    state_input_layer = tf.keras.layers.Input(shape=(1 + observation_radius * 2, 1 + observation_radius * 2))
    message_input_layer = tf.keras.layers.Input(shape=(leader_message_size,))

    # Flatten spatial grid to sequence: (batch_size, time_steps = flattened size, features = 1)
    state_reshaped = tf.keras.layers.Reshape((-1, 1))(state_input_layer)
    message_reshaped = tf.keras.layers.Reshape((-1, 1))(message_input_layer)

    # Combine state and message inputs (NO value_input_layer anymore)
    combined_input = tf.keras.layers.Concatenate(axis=1)([state_reshaped, message_reshaped])

    x = tf.keras.layers.GRU(64, activation='relu', return_sequences=False)(combined_input)
    x = tf.keras.layers.Dense(64, activation='relu')(x)

    action_logits = tf.keras.layers.Dense(len(ACTION_SPACE), activation='linear', name="action_logits")(x)

    # Model
    follower_model = tf.keras.Model(inputs=[state_input_layer, message_input_layer], outputs=[action_logits])
    follower_model.summary()
    return follower_model
follower_model = build_follower_model(observation_radius, LEADER_MESSAGE_SIZE)


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 5, 5)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_2       │ (None, 8)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_1 (Reshape) │ (None, 25, 1)     │          0 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_2 (Reshape) │ (None, 8, 1)      │          0 │ input_layer_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 33, 1)     │          0 │ reshape_1[0][0],  │
│ (Concatenate)       │                   │            │ reshape_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gru_1 (GRU)         │ (None, 64)        │     12,864 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 64)        │      4,160 │ gru_1[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ action_logits       │ (None, 9)         │        585 │ dense_2[0][0]     │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 17,609 (68.79 KB)

 Trainable params: 17,609 (68.79 KB)

 Non-trainable params: 0 (0.00 B)

### Try to Run the Training

In [7]:
def generate_grid():
    # Sample Grid Map
    grid_map = np.zeros((10, 10), dtype=int)
    
    # Place 2 agents
    agent_positions = []
    while len(agent_positions) < 2:
        x, y = np.random.randint(0, 10, size=2)
        if grid_map[x, y] == FREE:
            grid_map[x, y] = AGENT
            agent_positions.append((x, y))
    
    # Place 1 target
    while True:
        x, y = np.random.randint(0, 10, size=2)
        if grid_map[x, y] == FREE:
            grid_map[x, y] = TARGET
            break

    # Place 5 soft obstacles
    for _ in range(5):
        while True:
            x, y = np.random.randint(0, 10, size=2)
            if grid_map[x, y] == FREE:
                grid_map[x, y] = OBSTACLE_SOFT
                break

    # Place 5 hard obstacles
    for _ in range(5):
        while True:
            x, y = np.random.randint(0, 10, size=2)
            if grid_map[x, y] == FREE:
                grid_map[x, y] = OBSTACLE_HARD
                break

    return grid_map

print("10x10 Grid Map:")
grid_map = generate_grid()
print(grid_map)

10x10 Grid Map:
[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 3 0]
 [0 0 0 0 0 0 0 0 0 2]
 [0 0 0 0 0 0 0 0 1 0]
 [0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 4 0]
 [0 2 0 0 0 1 3 1 2 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 2 0 0 1 2 0 0 0 0]]


In [8]:
def get_agents_info(grid_map):
    # Defining Leader vs Follower agents
    # Randomly select positions for leader and follower agents from the grid map
    agent_positions = np.argwhere(grid_map == AGENT)
    if len(agent_positions) > 0:
        leader_position = agent_positions[0]  # Assign the first agent position to the leader
        follower_position = agent_positions[1] if len(agent_positions) > 1 else None  # Assign the second agent position to the follower if available
    else:
        leader_position = None
        follower_position = None

    leader_agent = {'position': leader_position, 'role': 'leader'}
    follower_agent = {'position': follower_position, 'role': 'follower'}
    return leader_agent, follower_agent

leader_agent, follower_agent = get_agents_info(grid_map)

print("Leader Agent Position:", leader_agent['position'])
print("Follower Agent Position:", follower_agent['position'])

Leader Agent Position: [1 8]
Follower Agent Position: [7 6]


In [9]:
state_leader = grid_map[leader_agent['position'][0], leader_agent['position'][1]]
state_follower = grid_map[follower_agent['position'][0], follower_agent['position'][1]]
# Extract partial observability for the leader
x, y = leader_agent['position']
leader_partial_observability = grid_map[
    max(0, x - observation_radius):min(grid_map.shape[0], x + observation_radius + 1),
    max(0, y - observation_radius):min(grid_map.shape[1], y + observation_radius + 1)
]
# Extract partial observability for the follower
x, y = follower_agent['position']
follower_partial_observability = grid_map[
    max(0, x - observation_radius):min(grid_map.shape[0], x + observation_radius + 1),
    max(0, y - observation_radius):min(grid_map.shape[1], y + observation_radius + 1)
]


print("Leader Partial Observability:")
print(leader_partial_observability)
print("Follower Partial Observability:")
print(follower_partial_observability)

Leader Partial Observability:
[[0 0 0 0]
 [0 0 3 0]
 [0 0 0 2]
 [0 0 1 0]]
Follower Partial Observability:
[[0 0 0 0 0]
 [0 0 0 0 4]
 [0 1 3 1 2]
 [0 0 0 0 0]
 [1 2 0 0 0]]


In [10]:
# Pad the observation in case some parts are out of the grid
def pad_observation(obs, target_shape=(5, 5)):
    h, w = obs.shape
    target_h, target_w = target_shape

    pad_height = max(target_h - h, 0)
    pad_width = max(target_w - w, 0)

    pad_top = pad_height // 2
    pad_bottom = pad_height - pad_top
    pad_left = pad_width // 2
    pad_right = pad_width - pad_left

    padded_obs = np.pad(obs, ((pad_top, pad_bottom), (pad_left, pad_right)), mode='constant', constant_values=0)
    return padded_obs
padded_leader_obs = pad_observation(leader_partial_observability, target_shape=(1 + observation_radius * 2, 1 + observation_radius * 2))
padded_follower_obs = pad_observation(follower_partial_observability, target_shape=(1 + observation_radius * 2, 1 + observation_radius * 2))

In [11]:
class MAPPO:
    def __init__(self, leader_model, follower_model, lr=0.001):
        self.leader_model = leader_model
        self.follower_model = follower_model
        self.lr = lr

    @staticmethod
    def contrastive_loss(messages, positive_pairs, temperature=0.1):
        """
        Compute the contrastive loss for communication alignment.

        Parameters:
        - messages: Tensor of shape (batch_size, embedding_dim), normalized embeddings.
        - positive_pairs: List of indices representing positive pairs.
        - temperature: Temperature parameter for scaling the similarity matrix.

        Returns:
        - loss: Contrastive loss value.
        """
        # Normalize the embeddings
        messages = tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1))(messages)
        # Compute the similarity matrix
        sim_matrix = tf.matmul(messages, messages, transpose_b=True) / temperature
        sim_matrix = tf.reshape(sim_matrix, (-1, 1))
        # Create one-hot labels for positive pairs
        labels = tf.one_hot(positive_pairs, depth=len(messages))
        labels = tf.reshape(labels, (-1, 1))

        # Compute the binary cross-entropy loss
        print("labels", labels)
        print("sim_matrix", sim_matrix)
        loss = tf.keras.losses.binary_crossentropy(y_true=labels, y_pred=sim_matrix, from_logits=True)
        return tf.reduce_mean(loss)

    def compute_loss(self, leader_partial_observability, follower_partial_observability, leader_message, action_logits, actions, value, reward, hyperparams: dict = None):
        '''
            Compute the loss for the MAPPO algorithm.

            Args:
                leader_partial_observability: Observations from the leader agent.
                follower_partial_observability: Observations from the follower agent.
                leader_message: Message sent by the leader agent.
                action_logits: Action logits from the model.
                actions: True actions taken by the agents.
                reward: Rewards received by the agents.
                value: Predicted value from the model.
                hyperparams: Hyperparameters for the loss function.
                
            Returns:
                loss: Computed loss value.
        '''
        # Hyperparameters
        contrastive_weight = 0.5  # Default value
        reconstruction_loss_weight = 0.2  # Default value
        entropy_bonus_weight = 0.01  # Default value
        if hyperparams:
            contrastive_weight = hyperparams.get('contrastive_weight', contrastive_weight)
            reconstruction_loss_weight = hyperparams.get('reconstruction_loss_weight', reconstruction_loss_weight)
            entropy_bonus_weight = hyperparams.get('entropy_bonus_weight', entropy_bonus_weight)
        
        leader_state = leader_partial_observability
        follower_state = follower_partial_observability

        # Compute Advantage (A = R + γV(s') - V(s))
        value = reward - value  # Predicted value
        advantage = reward - value  # TD error as Advantage Estimate
        print("loss", advantage)

         # Policy Gradient Loss (A2C)
        action_prob_leader = action_logits
        safe_action_prob_leader = tf.clip_by_value(action_prob_leader, 1e-8, 1.0)
        policy_loss = -tf.reduce_mean(advantage * tf.math.log(safe_action_prob_leader))
        print('Policy Gradient Loss', policy_loss)

        follower_state_reshaped = tf.expand_dims(tf.convert_to_tensor(follower_partial_observability, dtype=tf.float32), axis=0)
        follower_state_reshaped = tf.reshape(follower_state_reshaped, (1, 1 + observation_radius * 2, 1 + observation_radius * 2))
        action_prob_follower = self.follower_model([follower_state_reshaped, leader_message], training=True)

        # Contrastive Loss (CACL) for Communication Alignment
        contrastive_loss_value = tf.clip_by_value(
            self.contrastive_loss(tf.convert_to_tensor([action_prob_follower]), positive_pairs=[0]),
            clip_value_min=0.0,
            clip_value_max=10.0
        )
        print('Contrastive Loss', contrastive_loss_value)

        # Message Reconstruction Loss (L_recon)
        print(f'leader_message={action_prob_leader}')
        print(f'decoded_message= {action_prob_follower}')

        # Align shapes of leader_message and decoded_message
        min_dim = min(action_prob_leader.shape[-1], action_prob_follower.shape[-1])
        leader_message_aligned = action_prob_leader[..., :min_dim]
        decoded_message_aligned = action_prob_follower[..., :min_dim]

        # CCompute reconstruction loss
        reconstruction_loss = tf.clip_by_value(
            tf.reduce_mean(tf.keras.losses.MSE(leader_message_aligned, decoded_message_aligned)),
            clip_value_min=0.0,
            clip_value_max=10.0
        )
        print('Reconstruction Loss', reconstruction_loss)

        # Entropy Bonus for Exploration
        # Clip the entropy bonus to avoid extreme values
        entropy_bonus = tf.clip_by_value(
            -tf.reduce_mean(action_prob_leader * tf.math.log(action_prob_leader + 1e-8)),
            clip_value_min=0.0,
            clip_value_max=10.0
        )
        print('Entropy Bonus', entropy_bonus)

        # Final loss function
        total_loss = policy_loss + entropy_bonus_weight * entropy_bonus + contrastive_weight * contrastive_loss_value + reconstruction_loss_weight * reconstruction_loss
        print('Total Loss', total_loss)
        
        return total_loss
    
    def apply_gradients(self, state_leader, decoded_msg, action_leader, action_follower, reward, leader_message, encoded_message, decoded_message):
        with tf.GradientTape() as tape:
            loss = self.compute_loss(
                state_leader=state_leader,
                decoded_msg=decoded_msg,
                action_leader=action_leader,
                action_follower=action_follower,
                reward=reward,
                leader_message=leader_message,
                encoded_message=encoded_message,
                decoded_message=decoded_message
            )
        grads = tape.gradient(loss, self.leader_model.trainable_variables + self.follower_model.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.leader_model.trainable_variables + self.follower_model.trainable_variables))


In [12]:
def train_MAPPO(episodes, leader_model, follower_model, env, hyperparams: dict = None, algorithm="MAPPO"):
    leader_model.compile()
    follower_model.compile()
    
    print("Starting training...")
    # Logging
    episode_logs = []

    # Hyperparameters
    lr = hyperparams.get('lr', 0.001) if hyperparams else 0.001
    max_step_per_episode = hyperparams.get('max_steps', 100) if hyperparams else 100
    max_episodes = hyperparams.get('max_episodes', 100) if hyperparams else 100

    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

    # Initialize MAPPO model
    mappo_model = MAPPO(
        leader_model=leader_model,
        follower_model=follower_model,
        lr=lr
    )

    episodes = episodes or max_episodes
    for episode in range(episodes):
        print(f"\nEpisode {episode + 1}/{episodes}")

        # Reset environment
        grid_map = generate_grid()
        leader_pos = get_agents_info(grid_map)[0]['position']
        follower_pos = get_agents_info(grid_map)[1]['position']
        target_pos = np.argwhere(grid_map == TARGET)[0]

        total_reward = 0
        tether_violated, collisions = 0, 0
        distances = []

        episode_log = {"episode": episode + 1} # human readable episode number

        with tf.GradientTape(persistent=True) as tape:
            for step in range(max_step_per_episode):
                # === Leader ===
                x, y = leader_pos

                leader_obs_input = tf.expand_dims(
                    tf.reshape(tf.convert_to_tensor(padded_leader_obs.astype(np.float32)),
                            (padded_leader_obs.shape[0], padded_leader_obs.shape[1])),
                    axis=0
                )
                print("leader_obs_input", leader_obs_input.shape)

                action_logits_leader, message, value_prediction = leader_model(leader_obs_input, training=True)
                action_prob_leader = tf.nn.softmax(action_logits_leader)
                leader_action_idx = tf.argmax(action_prob_leader[0]).numpy()
                leader_action = list(ACTION_SPACE)[leader_action_idx]

                # Update leader position
                new_leader_pos = [leader_pos[0] + leader_action.value[0], leader_pos[1] + leader_action.value[1]]
                if 0 <= new_leader_pos[0] < grid_map.shape[0] and 0 <= new_leader_pos[1] < grid_map.shape[1]:
                    grid_map[leader_pos[0], leader_pos[1]] = FREE
                    leader_pos = new_leader_pos
                    grid_map[leader_pos[0], leader_pos[1]] = AGENT
                else:
                    print("Leader move out of bounds. Staying in place.")

                # === Follower ===
                follower_obs_input = tf.expand_dims(
                    tf.reshape(tf.convert_to_tensor(padded_follower_obs.astype(np.float32)),
                            (padded_follower_obs.shape[0], padded_follower_obs.shape[1])),
                    axis=0
                )
                print("follower_obs_input", follower_obs_input.shape)

                follower_action_logits = follower_model([follower_obs_input, message], training=True)
                action_prob_follower = tf.nn.softmax(follower_action_logits)
                follower_action_idx = tf.argmax(action_prob_follower[0]).numpy()
                follower_action = list(ACTION_SPACE)[follower_action_idx]

                # Update follower position
                new_follower_pos = [follower_pos[0] + follower_action.value[0], follower_pos[1] + follower_action.value[1]]
                if 0 <= new_follower_pos[0] < grid_map.shape[0] and 0 <= new_follower_pos[1] < grid_map.shape[1]:
                    grid_map[follower_pos[0], follower_pos[1]] = FREE
                    follower_pos = new_follower_pos
                    grid_map[follower_pos[0], follower_pos[1]] = AGENT
                else:
                    print("Follower move out of bounds. Staying in place.")

                # === Compute reward ===
                distance = np.linalg.norm(np.array(leader_pos) - np.array(follower_pos))
                distances.append(distance)

                step_reward = -1  # step penalty
                if distance > 2 or distance < 1:
                    tether_violated += 1
                if np.array_equal(leader_pos, target_pos) or np.array_equal(follower_pos, target_pos):
                    step_reward += REWARDS.TARGET.value
                if (grid_map[leader_pos[0], leader_pos[1]] == OBSTACLE_HARD or
                        grid_map[follower_pos[0], follower_pos[1]] == OBSTACLE_HARD):
                    step_reward += REWARDS.CRASH.value
                    collisions += 1

                total_reward += step_reward

                # === Compute loss ===
                loss = mappo_model.compute_loss(
                    leader_partial_observability=tf.reshape(leader_obs_input, (1, -1)),
                    follower_partial_observability=tf.reshape(follower_obs_input, (1, -1)),
                    leader_message=message,
                    action_logits=action_logits_leader,
                    actions=[leader_action_idx, follower_action_idx],
                    value=value_prediction,
                    reward=tf.convert_to_tensor([step_reward], dtype=tf.float32),
                    hyperparams=None
                )

            # === After episode: Backpropagate ===
            grads = tape.gradient(loss, leader_model.trainable_variables + follower_model.trainable_variables)
            optimizer.apply_gradients(zip(grads, leader_model.trainable_variables + follower_model.trainable_variables))

        # === Logging ===
        avg_distance = np.mean(distances) if distances else 0
        entropy_bonus = -tf.reduce_mean(action_prob_leader * tf.math.log(action_prob_leader + 1e-8))
        contrastive_loss = float(mappo_model.contrastive_loss(messages=tf.convert_to_tensor([action_prob_follower]), positive_pairs=[0]))

        print(f"Episode {episode + 1}: Total Reward = {total_reward}")

        episode_log.update({
            "reward": total_reward,
            "avg_distance": avg_distance,
            "policy_loss": float(loss),
            "contrastive_loss": contrastive_loss,
            "entropy": float(entropy_bonus),
            "tether_violations": tether_violated,
            "collisions": collisions,
            "hyperparams": hyperparams,
            "algorithm": algorithm,
            "timestamp": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "num_episodes": episodes
        })

        episode_logs.append(episode_log)

    # === Export logs ===
    logs_df = pd.DataFrame(episode_logs)
    os.makedirs('logs', exist_ok=True)
    logs_df.to_csv("logs/evaluation_metrics.csv", index=False)
    print(f"Training logs exported to 'evaluation_metrics.csv'")
    print("✅Training completed.")

# Call the function
train_MAPPO(
    episodes=1,
    leader_model=leader_model,
    follower_model=follower_model,
    env=None,  # Placeholder for the environment
    hyperparams={
        'lr': 0.001,
        'max_steps': 100,
        'max_episodes': 1
    },
    algorithm="MAPPO"
)

Starting training...

Episode 1/1
leader_obs_input (1, 5, 5)
follower_obs_input (1, 5, 5)
loss tf.Tensor([[-0.0352338]], shape=(1, 1), dtype=float32)
Policy Gradient Loss tf.Tensor(-0.53983694, shape=(), dtype=float32)
labels tf.Tensor([[1.]], shape=(1, 1), dtype=float32)
sim_matrix tf.Tensor([[89.99999]], shape=(1, 1), dtype=float32)
Contrastive Loss tf.Tensor(0.0, shape=(), dtype=float32)
leader_message=[[-0.02156862 -0.00935218 -0.00076089 -0.0368416  -0.01761251  0.01170359
  -0.0483242   0.0110927  -0.01374189]]
decoded_message= [[ 0.00261947  0.05250008  0.04822424 -0.0317661   0.02526565  0.02426799
  -0.02327193 -0.03178857  0.01627291]]
Reconstruction Loss tf.Tensor(0.0013555309, shape=(), dtype=float32)
Entropy Bonus tf.Tensor(nan, shape=(), dtype=float32)
Total Loss tf.Tensor(nan, shape=(), dtype=float32)
leader_obs_input (1, 5, 5)
Leader move out of bounds. Staying in place.
follower_obs_input (1, 5, 5)
loss tf.Tensor([[-0.0352338]], shape=(1, 1), dtype=float32)
Policy Grad

In [13]:
leader_model.save('leader_model.h5')
follower_model.save('follower_model.h5')

## Make Prediction on an Action

In [14]:
def predict_leader_action(leader_model, leader_obs_input):
    leader_obs_input = tf.convert_to_tensor(leader_obs_input, dtype=tf.float32)
    leader_obs_input = tf.expand_dims(tf.reshape(leader_obs_input, (leader_obs_input.shape[0], leader_obs_input.shape[1])), axis=0)

    action_logits_leader, message, value_prediction = leader_model(leader_obs_input, training=False)
    action_prob_leader = tf.nn.softmax(action_logits_leader)
    action_idx = tf.argmax(action_prob_leader, axis=-1).numpy()[0]
    action = list(ACTION_SPACE)[action_idx]
    return action, message, value_prediction


In [15]:
def predict_follower_action(follower_model, follower_obs_input, leader_message):
    # Ensure input shapes
    follower_obs_input = tf.convert_to_tensor(follower_obs_input, dtype=tf.float32)
    follower_obs_input = tf.expand_dims(tf.reshape(follower_obs_input, (follower_obs_input.shape[0], follower_obs_input.shape[1])), axis=0)

    leader_message = tf.convert_to_tensor(leader_message, dtype=tf.float32)
    leader_message = tf.expand_dims(leader_message, axis=0) if len(leader_message.shape) == 1 else leader_message

    action_logits_follower = follower_model([follower_obs_input, leader_message], training=False)
    action_prob_follower = tf.nn.softmax(action_logits_follower)
    action_idx = tf.argmax(action_prob_follower, axis=-1).numpy()[0]
    action = list(ACTION_SPACE)[action_idx]
    return action


In [16]:
# Leader step
leader_action, leader_message, leader_value = predict_leader_action(leader_model, padded_leader_obs)

# Follower step
follower_action = predict_follower_action(follower_model, padded_follower_obs, leader_message)

print("Leader Action:", leader_action.name, leader_action.value)
print("Follower Action:", follower_action.name, follower_action.value)

Leader Action: UP (-1, 0)
Follower Action: UP (-1, 0)


## Put the Agents into Play

In [17]:
def is_valid_move(position, grid_map, leader_pos, follower_pos, tether_limit=2):
    x, y = position
    # Check bounds and hard obstacles
    if not (0 <= x < grid_map.shape[0] and 0 <= y < grid_map.shape[1]) or grid_map[x, y] == OBSTACLE_HARD:
        return False
    # Check if position hits another agent
    if np.array_equal(position, leader_pos) or np.array_equal(position, follower_pos):
        return False
    # Check tether limit
    if np.floor(np.sqrt((position[0] - leader_pos[0])**2 + (position[1] - leader_pos[1])**2)) > tether_limit:
        return False
    return True

In [24]:
def play_episode(leader_model, follower_model, max_steps=100):
    print("\n Starting Play Mode Episode")
    
    # Reset environment
    grid_map = generate_grid()
    leader_pos = np.array(get_agents_info(grid_map)[0]['position'])
    follower_pos = np.array(get_agents_info(grid_map)[1]['position'])
    target_pos = np.argwhere(grid_map == TARGET)[0]

    leader_path = [leader_pos.copy()]
    follower_path = [follower_pos.copy()]
    total_reward = 0

    for step in range(max_steps):
        print(f"\nStep {step + 1}/{max_steps}")

        # === Leader Prediction ===
        leader_obs = grid_map[
            max(0, leader_pos[0] - observation_radius):min(grid_map.shape[0], leader_pos[0] + observation_radius + 1),
            max(0, leader_pos[1] - observation_radius):min(grid_map.shape[1], leader_pos[1] + observation_radius + 1)
        ]
        padded_leader_obs = pad_observation(leader_obs, (1 + observation_radius * 2, 1 + observation_radius * 2))
        leader_action, leader_message, _ = predict_leader_action(leader_model, padded_leader_obs)

        print(f"Leader Action: {leader_action}, Leader Pos: {leader_pos}")

        # Update leader position
        new_leader_pos = [leader_pos[0] + leader_action.value[0], leader_pos[1] + leader_action.value[1]]
        if is_valid_move(new_leader_pos, grid_map, leader_pos, follower_pos):
            grid_map[leader_pos[0], leader_pos[1]] = FREE
            leader_pos = new_leader_pos
            grid_map[leader_pos[0], leader_pos[1]] = AGENT
        else:
            print("Invalid move for leader. Resetting game.")
            return play_episode(leader_model, follower_model, max_steps)
        leader_path.append(leader_pos.copy())

        # === Follower Prediction ===
        follower_obs = grid_map[
            max(0, follower_pos[0] - observation_radius):min(grid_map.shape[0], follower_pos[0] + observation_radius + 1),
            max(0, follower_pos[1] - observation_radius):min(grid_map.shape[1], follower_pos[1] + observation_radius + 1)
        ]
        padded_follower_obs = pad_observation(follower_obs, (1 + observation_radius * 2, 1 + observation_radius * 2))
        follower_action = predict_follower_action(follower_model, padded_follower_obs, leader_message)

        print(f"Follower Action: {follower_action}, Follower Pos: {follower_pos}")

        # Update follower position
        new_follower_pos = [follower_pos[0] + follower_action.value[0], follower_pos[1] + follower_action.value[1]]
        if is_valid_move(new_follower_pos, grid_map, leader_pos, follower_pos):
            grid_map[follower_pos[0], follower_pos[1]] = FREE
            follower_pos = new_follower_pos
            grid_map[follower_pos[0], follower_pos[1]] = AGENT
        else:
            print("Invalid move for follower. Resetting game.")
            return play_episode(leader_model, follower_model, max_steps)
        follower_path.append(follower_pos.copy())

        # Check termination
        if np.array_equal(leader_pos, target_pos) or np.array_equal(follower_pos, target_pos):
            print("Target reached!")
            break

        total_reward -= 1  # Step penalty

    print(f"\n✅ Episode finished. Total Reward: {total_reward}")
    print(f"Leader Path: {leader_path}")
    print(f"Follower Path: {follower_path}")

    return leader_path, follower_path

In [25]:
play_episode(leader_model, follower_model, max_steps=100)


 Starting Play Mode Episode

Step 1/100
Leader Action: ACTION_SPACE.UP, Leader Pos: [5 5]
Follower Action: ACTION_SPACE.UP, Follower Pos: [8 9]
Invalid move for follower. Resetting game.

 Starting Play Mode Episode

Step 1/100
Leader Action: ACTION_SPACE.UP, Leader Pos: [1 6]
Follower Action: ACTION_SPACE.UP, Follower Pos: [5 7]
Invalid move for follower. Resetting game.

 Starting Play Mode Episode

Step 1/100
Leader Action: ACTION_SPACE.UP, Leader Pos: [1 4]
Leader Action: ACTION_SPACE.UP, Leader Pos: [1 6]
Follower Action: ACTION_SPACE.UP, Follower Pos: [5 7]
Invalid move for follower. Resetting game.

 Starting Play Mode Episode

Step 1/100
Leader Action: ACTION_SPACE.UP, Leader Pos: [1 4]
Follower Action: ACTION_SPACE.UP, Follower Pos: [5 2]
Invalid move for follower. Resetting game.

 Starting Play Mode Episode

Step 1/100
Leader Action: ACTION_SPACE.UP, Leader Pos: [2 6]
Follower Action: ACTION_SPACE.UP, Follower Pos: [8 6]
Invalid move for follower. Resetting game.

 Starting

([array([6, 2]), [np.int64(5), np.int64(2)]],
 [array([8, 4]), [np.int64(7), np.int64(4)]])